<a href="https://colab.research.google.com/github/kangmg/compchem_with_colab/blob/main/ase_xtb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# clear cell output
from IPython.display import clear_output

In [2]:
# xtb installation
!wget -q https://github.com/grimme-lab/xtb/releases/download/v6.6.1/xtb-6.6.1-linux-x86_64.tar.xz -O ./xtb.tar.xz
!tar vxf ./xtb.tar.xz
!rm ./xtb.tar.xz
!mkdir xtb_tmp
clear_output()
!echo "xtb successfully installed!"

xtb downloaded!


In [3]:
!pip -q install git+https://github.com/cclib/cclib.git    # compchem package
!pip -q install git+https://gitlab.com/ase/ase.git        # ase
!pip -q install git+https://github.com/kangmg/xtb_ase.git # ase_xtb

clear_output()
!echo "dependent packaged are successfully installed!"

In [ ]:
# xtb enviromental setting
import os
os.environ['PATH'] += ':/content/xtb-6.6.1/bin'

In [34]:
aspirin = """21

C          2.45639       -0.16543        0.39727
C          0.78890        2.02854       -0.64816
C          2.74071        1.02327        0.12533
C          2.05734        2.11782       -0.40016
C         -3.59396        1.03209        0.04589
C          0.67738       -0.27507        0.15097
C          0.00274        0.84234       -0.35887
O          0.63293       -2.54526        0.93360
O         -1.76452        1.43252        1.56571
O         -1.24894       -1.68411        0.12975
C          0.05278       -1.58881        0.44958
C         -2.15199        1.12841        0.44513
O         -1.36167        0.82832       -0.65825
H         -1.47652       -2.60111        0.39232
H          2.60948       -1.11173        0.80284
H          0.15708        2.88266       -1.05823
H          3.80777        1.09126        0.32364
H          2.59044        3.04081       -0.61405
H         -3.81640        0.02049       -0.30287
H         -4.22480        1.24591        0.91440
H         -3.81296        1.76576       -0.73332"""

In [36]:
# utils
from io import StringIO

# ase / xtb
import ase
from ase.io import read
from xtb_ase import XTB
from ase.optimize import BFGS


aspirin_mol:ase.Atoms = read(StringIO(aspirin), format="xyz")

aspirin_mol.calc = XTB(m,)

print(f"pre-opt : {aspirin_mol.get_potential_energy()}\n")

optimizer = BFGS(aspirin_mol)

optimizer.run()

print("\nopt : ", aspirin_mol.get_potential_energy())

pre-opt : -39.519190020128

      Step     Time          Energy          fmax
BFGS:    0 19:48:32      -39.519190       10.991238
BFGS:    1 19:48:33      -39.581955        8.030148
BFGS:    2 19:48:33      -39.608381        3.840010
BFGS:    3 19:48:33      -39.619962        1.675427
BFGS:    4 19:48:33      -39.623319        0.997034
BFGS:    5 19:48:33      -39.626236        0.730927
BFGS:    6 19:48:33      -39.626800        0.477471
BFGS:    7 19:48:33      -39.627223        0.245073
BFGS:    8 19:48:33      -39.627442        0.287164
BFGS:    9 19:48:33      -39.627640        0.158353
BFGS:   10 19:48:33      -39.627726        0.141911
BFGS:   11 19:48:33      -39.627771        0.112625
BFGS:   12 19:48:33      -39.627812        0.100507
BFGS:   13 19:48:33      -39.627843        0.074500
BFGS:   14 19:48:33      -39.627863        0.038365

opt :  -39.627863282763


In [49]:
# utils
from io import StringIO
from IPython.display import clear_output

# ase / xtb
import ase
from ase.io import read
from xtb_ase import XTB
from ase.optimize import BFGS

def xtb_optimize(mol:str|ase.Atoms, charge:int, method:str="gfn2-xTB", spinpol:bool|None=None, uhf:int=0, clear_log=False):
  """
  Description
  -----------
  xtb geometry optimize 함수

  Parameters
  ----------
  mol (xyz format string | ase.Atoms) :
  charge (int) :
  method (str) :
  spinpol ( bool | None ) :
  uhf (int) :
  tmp_dir (str) :

  Returns
  -------
  optimized xyz (str) :
  """

  # convert xyz format --> ase.Atoms
  if type(mol) == str:
    try:
      mol = ase.io.read(StringIO(mol), format="xyz")
    except:
      raise ValueError("Invalid xyz format string")

  # set XTB calculator
  mol.calc = XTB(method=method, charge=charge, spinpol=spinpol, uhf=uhf)
  # geometry optimize
  optimizer = BFGS(mol)
  optimizer.run()

  # get xyz format string
  with StringIO() as output:
    ase.io.write(output, mol, format="xyz")
    opt_xyz = output.getvalue()

  # clear cell output
  if clear_log:
    clear_output()

  return opt_xyz

In [52]:
print(xtb_optimize(aspirin, 0, clear_log=True))

21

C       2.096257843916880     -0.164742044176155      0.374260564243741
C       0.723261465970201      2.009086403739904     -0.633950159649945
C       2.771656024519402      1.010376090280658      0.117628565586992
C       2.083962227726616      2.097636023710908     -0.397669307214821
C      -3.587874959095477      1.027843734912582      0.047990427678304
C       0.723255779519428     -0.273081225858182      0.147341093220439
C       0.035605802066217      0.836495136448088     -0.353957312491734
O       0.698940182050824     -2.510017358016377      0.939377639050611
O      -1.735926659475403      1.417268049992791      1.536900413331231
O      -1.200474609949215     -1.670412890191387      0.144589751722368
C       0.098487044162888     -1.581252182425427      0.459017266945291
C      -2.142027365043912      1.112465686131596      0.450792126801404
O      -1.334229611245795      0.809308901592228     -0.586786616034317
H      -1.527265720903839     -2.555850255383092      0.3701